In [12]:
import cv2
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk, ImageEnhance, ImageFilter
import face_recognition
import numpy as np
import threading
import time


class FaceDetectionTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Advanced Face Detection Enhancement Tool")
        self.root.minsize(1000, 1000)
        self.image_path = None
        self.original_image = None
        self.processed_image = None
        self.face_detected = False
        self.stop_calibration = False
        self.confidence = 0.0
        self.minimum_confidence = 80.0  # Target confidence percentage

        # Default enhancement parameters
        self.settings = {
            "brightness": 1.0,
            "sharpness": 1.0,
            "contrast": 1.0,
            "hue": 0,
            "saturation": 1.0,
            "gamma": 1.0,
            "blur": 0,
            "edge_enhancement": 0,
            "shadows": 0.0,
            "highlights": 0.0,
        }

        # Create GUI elements
        self.create_gui()

    def create_gui(self):
        # Status labels
        self.status_label = tk.Label(self.root, text="Load an image to start.", font=("Arial", 14))
        self.status_label.pack(pady=10)

        self.confidence_label = tk.Label(self.root, text="Confidence: 0%", font=("Arial", 12))
        self.confidence_label.pack(pady=5)

        # Frame for image display
        self.image_frame = tk.Label(self.root)
        self.image_frame.pack(pady=10)

        # Buttons to load, save images, copy settings, and calibration
        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="Load Image", command=self.load_image).grid(row=0, column=0, padx=5, pady=5)
        tk.Button(button_frame, text="Copy Settings", command=self.copy_settings).grid(row=0, column=1, padx=5, pady=5)
        tk.Button(button_frame, text="Save Processed Image", command=self.save_image).grid(row=0, column=2, padx=5, pady=5)
        self.calibrate_button = tk.Button(button_frame, text="Start Calibration", command=self.start_calibration)
        self.calibrate_button.grid(row=0, column=3, padx=5, pady=5)

        # Scrollable settings panel
        settings_frame = tk.Frame(self.root)
        settings_frame.pack(fill=tk.BOTH, expand=True)

        canvas = tk.Canvas(settings_frame)
        scrollbar = ttk.Scrollbar(settings_frame, orient="vertical", command=canvas.yview)
        self.scrollable_frame = tk.Frame(canvas)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all")),
        )

        canvas.create_window((0, 0), window=self.scrollable_frame, anchor="n")
        canvas.configure(yscrollcommand=scrollbar.set)

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        # Enable mouse wheel scrolling
        self.scrollable_frame.bind("<Enter>", lambda e: self.bind_mouse_wheel(canvas))
        self.scrollable_frame.bind("<Leave>", lambda e: self.unbind_mouse_wheel(canvas))

        # Sliders for adjustments
        for setting, default in self.settings.items():
            frame = tk.Frame(self.scrollable_frame)
            frame.pack(pady=10, anchor="center", fill="x")

            tk.Label(frame, text=setting.capitalize()).pack(side="top", pady=5)
            slider = tk.Scale(
                frame,
                from_=0 if setting != "gamma" else 0.1,
                to=2 if setting != "hue" else 180,
                resolution=0.1,
                orient="horizontal",
                command=lambda val, s=setting: self.update_setting(s, val),
                length=800,
            )
            slider.set(default)
            slider.pack()

    def bind_mouse_wheel(self, canvas):
        canvas.bind_all("<MouseWheel>", lambda event: canvas.yview_scroll(-1 * (event.delta // 120), "units"))

    def unbind_mouse_wheel(self, canvas):
        canvas.unbind_all("<MouseWheel>")

    def resize_image(self, image, max_width=800, max_height=600):
        """Resize the image to fit within the specified max dimensions while maintaining aspect ratio."""
        height, width = image.shape[:2]
        scaling_factor = min(max_width / width, max_height / height)
        new_width = int(width * scaling_factor)
        new_height = int(height * scaling_factor)
        resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
        return resized_image

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.png;*.jpeg")])
        if not file_path:
            return
        self.image_path = file_path
        image = cv2.imread(self.image_path)
        self.original_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.original_image = self.resize_image(self.original_image)  # Resize large images
        self.processed_image = self.original_image.copy()
        self.apply_enhancements()

    def calculate_confidence(self, face_locations):
        """
        Confidence is calculated based on the proportion of the image covered by detected faces.
        """
        if not face_locations:
            return 0.0

        image_area = self.processed_image.shape[0] * self.processed_image.shape[1]
        face_areas = [
            (bottom - top) * (right - left)
            for top, right, bottom, left in face_locations
        ]
        total_face_area = sum(face_areas)
        return (total_face_area / image_area) * 100

    def display_image(self, image, face_locations=None):
        img = image.copy()
        if face_locations:
            for top, right, bottom, left in face_locations:
                cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(img)
        self.image_frame.imgtk = imgtk
        self.image_frame.configure(image=imgtk)

    def apply_enhancements(self):
        if self.original_image is None:
            return
        pil_image = Image.fromarray(self.original_image)

        # Apply enhancements
        pil_image = ImageEnhance.Brightness(pil_image).enhance(self.settings["brightness"])
        pil_image = ImageEnhance.Sharpness(pil_image).enhance(self.settings["sharpness"])
        pil_image = ImageEnhance.Contrast(pil_image).enhance(self.settings["contrast"])
        pil_image = ImageEnhance.Color(pil_image).enhance(self.settings["saturation"])

        # Apply hue adjustment
        if self.settings["hue"] != 0:
            hsv = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2HSV)
            hsv[..., 0] = (hsv[..., 0] + self.settings["hue"]) % 180
            pil_image = Image.fromarray(cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB))

        # Apply gamma correction
        if self.settings["gamma"] != 1.0:
            inv_gamma = 1.0 / self.settings["gamma"]
            lut = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
            pil_image = Image.fromarray(cv2.LUT(np.array(pil_image), lut))

        # Apply blur
        if self.settings["blur"] > 0:
            pil_image = pil_image.filter(ImageFilter.GaussianBlur(self.settings["blur"]))

        # Apply edge enhancement
        if self.settings["edge_enhancement"] > 0:
            for _ in range(int(self.settings["edge_enhancement"])):
                pil_image = pil_image.filter(ImageFilter.EDGE_ENHANCE)

        self.processed_image = np.array(pil_image)

        # Detect faces and calculate confidence
        face_locations = face_recognition.face_locations(self.processed_image)
        self.confidence = self.calculate_confidence(face_locations)

        # Update status and confidence label
        if len(face_locations) > 0:
            self.status_label.config(text=f"Face(s) Detected: {len(face_locations)}", fg="green")
        else:
            self.status_label.config(text="No Face Detected", fg="red")
        self.confidence_label.config(text=f"Confidence: {self.confidence:.2f}%")

        self.display_image(self.processed_image, face_locations)

    def update_setting(self, setting, value):
        self.settings[setting] = float(value)
        self.apply_enhancements()

    def copy_settings(self):
        settings = ", ".join(f"{key}: {value}" for key, value in self.settings.items())
        self.root.clipboard_clear()
        self.root.clipboard_append(settings)
        self.root.update()  # Keep the clipboard content
        messagebox.showinfo("Settings Copied", "Settings copied to clipboard!")

    def save_image(self):
        if self.processed_image is None:
            messagebox.showerror("Error", "No processed image to save.")
            return
        save_path = "processed_image.jpg"
        cv2.imwrite(save_path, cv2.cvtColor(self.processed_image, cv2.COLOR_RGB2BGR))
        messagebox.showinfo("Image Saved", f"Image saved to {save_path}")

    def start_calibration(self):
        self.stop_calibration = False
        self.calibrate_button.config(text="Stop Calibration", command=self.stop_calibration_process)
        threading.Thread(target=self.calibration_process).start()

    def stop_calibration_process(self):
        self.stop_calibration = True
        self.calibrate_button.config(text="Start Calibration", command=self.start_calibration)

    # Add this improved calibration_process() function to the existing script
    def calibration_process(self):
        parameters = ["brightness", "contrast", "sharpness", "saturation", "gamma"]
        step_sizes = {
            "brightness": 0.1,
            "contrast": 0.1,
            "sharpness": 0.1,
            "saturation": 0.1,
            "gamma": 0.1,
        }

        max_attempts_per_param = 20  # Limit iterations per parameter to prevent infinite loops
        improvement_threshold = 0.5  # Minimum confidence improvement to continue with the same parameter

        previous_confidence = self.confidence
        for param in parameters:
            attempts = 0
            while attempts < max_attempts_per_param and not self.stop_calibration:
                # Adjust the parameter up or down
                current_value = self.settings[param]
                new_value = current_value + step_sizes[param]
                if new_value > 2:  # Reset to the lower bound if out of range
                    new_value = 0.5

                self.settings[param] = new_value
                self.apply_enhancements()

                if self.confidence >= self.minimum_confidence:
                    self.stop_calibration_process()
                    return

                if self.confidence - previous_confidence < improvement_threshold:
                    attempts += 1  # No significant improvement, increment attempts
                else:
                    previous_confidence = self.confidence  # Confidence improved, reset attempts

            # After exhausting attempts on one parameter, reset it and move to the next
            self.settings[param] = 1.0  # Reset to default
            self.apply_enhancements()

        self.stop_calibration_process()


if __name__ == "__main__":
    root = tk.Tk()
    app = FaceDetectionTool(root)
    root.mainloop()
